CNN 

The Problem: MNIST digit classification
	
We’re going to tackle a classic introductory Computer Vision problem: MNIST handwritten digit classification. It’s simple: given an image, classify it as a digit
	
Each image in the MNIST dataset is 28x28 and contains a centered, grayscale digit. Our CNN will take an image and output one of 10 possible classes (one for each digit)

First of all,  import all necessary packages



In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import fashion_mnist #This dataset contains images of fashion
#It is almost completely similar to the regular mnist dataset, only difference is images are now fashion.
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

In [2]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Before we begin, we’ll normalize the image pixel values from [0, 255] to [-0.5, 0.5]
	 
This will  make our network easier to train (using smaller, centered values usually leads to better results). 

We’ll also reshape each image from (28, 28) to (28, 28, 1) because Keras requires the third dimension. (1 stands for color scheme, which is grey scale in this case). 



In [3]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

In [4]:
# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

Keras model is either built using the Sequential class, which represents a linear stack of layers, or the functional Model class, which is more customizable. 

We’ll be using the simpler Sequential model, since our CNN will be a linear stack of layers.

num_filters, filter_size, and pool_size are self-explanatory variables that set the hyperparameters for our CNN.

The first layer in any Sequential model must specify the input_shape, so we do so on Conv2D. 

Once this input shape is specified, Keras will automatically infer the shapes of inputs for later layers.
	
The output Softmax layer has 10 nodes, one for each class




In [5]:

num_filters = 10 #Originally 8 in mnist. 
filter_size = 3
pool_size = 2

In [6]:
# Build the model.
model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1)),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])

Before we can begin training, we need to configure the training process. We decide 3 key factors during the compilation step:
	
The optimizer. We’ll stick with a pretty good default: the Adam gradient-based optimizer. Keras has many other optimizers you can look into as well.

The loss function. Since we’re using a Softmax output layer, we’ll use the Cross-Entropy loss. Keras distinguishes between binary_crossentropy (2 classes) and categorical_crossentropy (>2 classes), so we’ll use the latter. 

A list of metrics. Since this is a classification problem, we’ll just have Keras report on the accuracy metric.



In [7]:
# Compile the model.
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

Training a model in Keras literally consists only of calling fit() and specifying some parameters. There are a lot of possible parameters, but we’ll only supply these:

The training data (images and labels), commonly known as X and Y, respectively.

The number of epochs (iterations over the entire dataset) to train for.

The validation data (or test data), which is used during training to periodically measure the network’s performance against data it hasn’t seen before.

There’s one thing we have to be careful about:

Keras expects the training targets to be 10-dimensional vectors, since there are 10 nodes in our Softmax output layer. Right now, our train_labels and test_labels arrays contain single integers representing the class for each image

Conveniently, Keras has a utility method that fixes this exact issue:
to_categorical: It turns our array of class integers into an array of one-hot vectors instead. For example, 2 would become [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] (it’s zero-indexed)





In [8]:
# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=50, #Original 30
  validation_data=(test_images, to_categorical(test_labels)),
)

Epoch 1/50
1875/1875 [==============================] - 8s 3ms/step - loss: 0.6653 - accuracy: 0.7759 - val_loss: 0.4206 - val_accuracy: 0.8480
Epoch 2/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3801 - accuracy: 0.8684 - val_loss: 0.3735 - val_accuracy: 0.8696
Epoch 3/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3407 - accuracy: 0.8819 - val_loss: 0.3489 - val_accuracy: 0.8764
Epoch 4/50
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3206 - accuracy: 0.8881 - val_loss: 0.3330 - val_accuracy: 0.8839
Epoch 5/50
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2993 - accuracy: 0.8957 - val_loss: 0.3406 - val_accuracy: 0.8789
Epoch 6/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2851 - accuracy: 0.8989 - val_loss: 0.3217 - val_accuracy: 0.8871
Epoch 7/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2728 - accuracy: 0.9029 - val_loss: 0.3271 - val_accuracy:

We pass an array of inputs to predict() and it returns an array of outputs.

The output of our network is 10 probabilities (because of softmax), so we’ll use np.argmax() to turn those into actual digits.


In [9]:
# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

In [10]:
# Print our model's predictions.
print("These are the model predictions :")
print(np.argmax(predictions, axis=1)) # [9 2 1 1 6]
print("\n")

These are the model predictions :
[9 2 1 1 6]




In [11]:
# Check our predictions against the ground truths.
print("These are the corresponding labels :")
print(test_labels[:5]) # [9 2 1 1 6]

These are the corresponding labels :
[9 2 1 1 6]
